In [2]:
import json 
import re
import cv2
import matplotlib.pyplot as plt

In [71]:
def bounding_box_to_segment(image, bb):
    x_coords = [point['x'] for point in bb]
    y_coords = [point['y'] for point in bb]

    # Determine the bounding box
    left = min(x_coords)
    top = min(y_coords)
    right = max(x_coords)
    bottom = max(y_coords)
    return image[top:bottom, left:right]

teeth_found = 0
azure_labels = []
img_i = 0
for i in range(1, 1000):
    file_path = f'/home/riikoro/fossil_data/Master Catalogues/Master Catalogue No.5-45624-53994/scan0210.pdf_pages/page{i}.json'
    pdf = cv2.imread(f'/home/riikoro/fossil_data/Master Catalogues/Master Catalogue No.5-45624-53994/scan0210.pdf_pages/page{i}.png')
    # pdf = cv2.cvtColor(pdf, cv2.COLOR_BGR2RGB) 
    # Open the JSON file
    try:
        with open(file_path, 'r') as json_file:
            # Load the JSON data into a Python object
            data = json.load(json_file)
    except FileNotFoundError:
        break

    for block in data['readResult']['blocks']:
        for line in block['lines']:
            # print(line['text'])
            for i, word in enumerate(line['words']):
                is_tooth = re.match('^[a-zA-Z]\d$|^[cC]$', word['text'])
                if (is_tooth):
                    teeth_found += 1
                    azure_label = word['text']
                    bounding_box = word['boundingPolygon']
                    azure_labels.append(azure_label)
                    img_segment = bounding_box_to_segment(pdf, bounding_box)
                    cv2.imwrite(f'/home/riikoro/fossil_data/tooth_samples/v4/{img_i}.png', img_segment)
                    img_i += 1

[ WARN:0@1886.152] global loadsave.cpp:241 findDecoder imread_('/home/riikoro/fossil_data/Master Catalogues/Master Catalogue No.5-45624-53994/scan0210.pdf_pages/page36.png'): can't open/read file: check file path/integrity


In [72]:
teeth_found

92

In [73]:
azure_labels

['M2',
 'M3',
 'M2',
 'P4',
 'M2',
 'M3',
 'M2',
 'M2',
 'w2',
 'M3',
 'M3',
 'M3',
 'M3',
 'C',
 'M3',
 'M6',
 'P3',
 'M3',
 'M3',
 'M3',
 'C',
 'L5',
 'C',
 'M5',
 'L5',
 'M2',
 'M3',
 'L5',
 'C',
 'T1',
 'P3',
 'p4',
 'M3',
 'M3',
 'M2',
 'm2',
 'P3',
 'm2',
 'm3',
 'M2',
 'P5',
 'P2',
 'P2',
 'M2',
 'M3',
 'P4',
 'P3',
 'm2',
 'M2',
 'M3',
 'P1',
 'M3',
 'M3',
 'P5',
 'M1',
 'M3',
 'M3',
 'M2',
 'P3',
 'p4',
 'C',
 'M2',
 'P3',
 'P4',
 'C',
 'C',
 'P4',
 'M3',
 'F1',
 'B1',
 'M3',
 'M1',
 'P2',
 'M2',
 'P4',
 'M2',
 'W1',
 'M1',
 'M3',
 'C',
 'M2',
 'M3',
 'C',
 'C',
 'M3',
 'C',
 'M2',
 'M2',
 'P3',
 'M2',
 'M2',
 'P1']

In [74]:
from functools import reduce
# get class balances found
count_m = reduce(lambda count, s: count + 1 if s.startswith(('m', 'M')) else count, azure_labels, 0)
count_p = reduce(lambda count, s: count + 1 if s.startswith(('p', 'P')) else count, azure_labels, 0)
count_i = reduce(lambda count, s: count + 1 if s.startswith(('i', 'I')) else count, azure_labels, 0)
count_c = reduce(lambda count, s: count + 1 if s.startswith(('c', 'C')) else count, azure_labels, 0)
print(f'm: {count_m}, p: {count_p}, i: {count_i}, c: {count_c}')

m: 52, p: 21, i: 0, c: 11


In [75]:
with open('/home/riikoro/fossil_data/tooth_samples/v4/azure_labels.txt', 'w') as out:
    out.write('\n'.join(azure_labels))